# HHI of Stargaze network

In [2]:
import json
from typing import List

Get the state of all validators.

In [3]:
!starsd query staking validators --node https://rpc.stargaze.ezstaking.io:443/ --output json > validators.json

Get the delegation associated with each validator.

In [6]:
validators = json.load(open('validators.json', 'r'))['validators']
tokens = [int(v['tokens']) for v in validators]

In [7]:
def values_to_marketshares (values: List[int]) -> List[float]:
    '''
    Takes value of each firm and turns it into a proportion, i.e., 
    their share of the total market.
    '''
    total = sum(values)
    return [v / total for v in values ]

assert(values_to_marketshares([10, 10, 30]) == [0.2, 0.2, 0.6])

In [8]:
def hhi (marketshares: List[float]):
    '''
    "The HHI is calculated by squaring the market share of each firm competing in the market 
    and then summing the resulting numbers."
    
    U.S. federal agencies "generally consider markets in which the HHI is between 
    1,500 and 2,500 points to be moderately concentrated, and consider markets 
    in which the HHI is in excess of 2,500 points to be highly concentrated."
    
    From https://www.justice.gov/atr/herfindahl-hirschman-index
    '''
    squares = [(x*100)*(x*100) for x in marketshares]
    return sum(squares)
    
assert(hhi([.30, .30, .20, .20]) == 2600)

In [9]:
hhi(values_to_marketshares(tokens))

156.48671026391222